# Рекомендательная система на библиотеке surprise (Практика 2.2)

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise```
или
```
$ conda install -c conda-forge scikit-surprise```

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from surprise import Dataset
from surprise import Reader

In [ ]:
data = Dataset.load_builtin('ml-100k')##загружем данные о фильмах

In [ ]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
df_vc=df.user.value_counts()

In [ ]:
df_vc[df_vc<30].index

Index(['706', '103', '80', '657', '672', '769', '146', '602', '696', '238',
       ...
       '571', '309', '475', '812', '147', '685', '364', '572', '631', '242'],
      dtype='object', length=199)

In [ ]:
train_df=df[~df.user.isin(list(df_vc[df_vc<30].index))]
test_df=df[df.user.isin(list(df_vc[df_vc<30].index))]

In [ ]:
#просто средние оценки (a)
mean_raitng=train_df.groupby(['item']).mean()
mra=mean_raitng.sort_values(['rating'], ascending=False)
mra

,rating
item,
1189,5.0
1293,5.0
1467,5.0
814,5.0
1500,5.0
...,...
1575,1.0
1340,1.0
852,1.0


In [ ]:
list(mra[0:5].index)

['1189', '1293', '1467', '814', '1500']

In [ ]:
test_df[test_df.item.isin(list(mra[0:5].index))]

,user,item,rating,timestamp
49046,146,1293,5.0,891458080


In [ ]:
df_vci=train_df.item.value_counts()
df_vci[df_vci>4].index

Index(['50', '100', '181', '1', '258', '174', '121', '288', '294', '56',
       ...
       '1301', '868', '1416', '1138', '1427', '104', '666', '1517', '1597',
       '1592'],
      dtype='object', length=1337)

In [ ]:
#средние оценки для фильмов, у которых больше 4 оценок (б)
mrb= mean_raitng[mean_raitng.index.isin(df_vci[df_vci>4].index)].sort_values(['rating'], ascending=False)
mrb

,rating
item,
1449,4.625000
169,4.482456
318,4.474048
408,4.472222
483,4.470085
...,...
1287,1.333333
424,1.315789
438,1.166667


In [ ]:
list(mrb[0:5].index)

['1449', '169', '318', '408', '483']

In [ ]:
test_df[test_df.item.isin(list(mrb[0:5].index))]

,user,item,rating,timestamp
14635,163,318,4.0,891220161
17767,172,483,3.0,875538028
21572,359,408,5.0,886453239
28736,512,318,5.0,888579569
30909,9,483,5.0,886960056
36085,202,318,1.0,879727116
36285,575,483,3.0,878148137
46457,516,169,5.0,891290685
55722,609,408,5.0,886896185
59059,212,318,5.0,879303928


In [ ]:
def MAP_at_n(df_ratings_all, n=10,threshold=5):
    users=df_ratings_all['userID'].unique()
    map_at_n=0
    for i in users:
        df_ratings=df_ratings_all[df_ratings_all.userID==i]
        map_at_n=map_at_n+Avg_Precision_at_n(df_ratings,n=n,threshold=threshold)
    return map_at_n/len(users)   